# Machine translation

In [102]:
!pip install -q datasets sentencepiece transformers torch transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2834.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


## Translation using transformer

In [103]:
pl_text = "Wczesnym rankiem, gdy słońce dopiero zaczynało wschodzić, ulice miasta były jeszcze puste. Czuć było świeżość powietrza i delikatny zapach kawy unoszący się z pobliskiej kawiarni. To był idealny moment na spokojny spacer przed rozpoczęciem dnia."
en_text = "Early in the morning, as the sun was just beginning to rise, the city streets were still empty. The air felt fresh, and a gentle smell of coffee drifted from a nearby café. It was the perfect moment for a peaceful walk before the day began."

In [104]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-pl-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-pl-en")

In [105]:
tokenized_text_pl = tokenizer(pl_text, return_tensors="pt")
translated_tokens = model.generate(input_ids=tokenized_text_pl["input_ids"], attention_mask=tokenized_text_pl['attention_mask'])
translated = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]


In [106]:
print(f"Polish text:\n {pl_text}")
print(f"Translated text:\n {translated}")
print(f"Expected translation:\n {en_text}")

Polish text:
 Wczesnym rankiem, gdy słońce dopiero zaczynało wschodzić, ulice miasta były jeszcze puste. Czuć było świeżość powietrza i delikatny zapach kawy unoszący się z pobliskiej kawiarni. To był idealny moment na spokojny spacer przed rozpoczęciem dnia.
Translated text:
 Early in the morning, when the sun was just beginning to rise, the streets of the city were still empty. It felt fresh air and a gentle smell of coffee floating from a nearby cafe. It was the perfect moment for a quiet walk before the beginning of the day.
Expected translation:
 Early in the morning, as the sun was just beginning to rise, the city streets were still empty. The air felt fresh, and a gentle smell of coffee drifted from a nearby café. It was the perfect moment for a peaceful walk before the day began.


## Language detection

In [107]:
!pip install -q langdetect

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2839.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [108]:
from langdetect import detect

source_text = "Az ezer mérföldes utazás is egyetlen lépéssel kezdődik."
print(f"Detected language: {detect(source_text)}")

Detected language: hu


## Language detection and translation with HERBERT model

In [109]:
!pip install -q googletrans==4.0.0-rc1 nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2844.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [110]:
import googletrans

text = """Szklanka ma u mnie pojemność 250 ml.
Do usmażenia naleśników użyłam szerokiej patelni do naleśników: średnica 24 cm.
Z podanej ilości składników wyszło mi 16 bardzo cienkich naleśników. 
Jajka wyjmij wcześniej z lodówki. Możesz też lekko podgrzać mleko do temperatury nie wyższej niż 40 stopni.

Kalorie policzone zostały na podstawie użytych przeze mnie składników. Jest to więc orientacyjna ilość kalorii, ponieważ nawet mąka może mieć inną ilość kalorii niż ta, której użyłam ja. 

Naleśniki szykuję przynajmniej raz w tygodniu. To danie proste, szybkie i uwielbiane przez dzieci. Możesz je podać na dowolny posiłek w ciągu dnia. Naleśniki możesz zabrać do pracy, do szkoły lub na piknik. Są po prostu boskie!""".replace("\n", " ")
translator = googletrans.Translator()

def translate(text, dest) -> str:
    return translator.translate(text, dest=dest, src="auto").text

destinations = {
    "en": "english",
    "es": "spanish",
    "de": "german",
    #"lt": "lithuanian",
    "ru": "russian",
    #"hu": "hungarian",
    "it": "italian",
    "fr": "french",
    #"sl": "slovenian",
    "no": "norwegian",
}
translated = []
for des in destinations.keys():
    translated.append(translate(text, des))

In [111]:
translated

['The glass has a capacity of 250 ml for me.To fry pancakes, I used a wide pan to pancakes: diameter 24 cm.I got 16 very thin pancakes from the amount of ingredients given.Take the eggs from the fridge beforehand.You can also lightly heat the milk to a temperature not higher than 40 degrees.Calories were counted on the basis of the ingredients used by me.So this is an approximate amount of calories, because even flour can have a different amount of calories than the one I used.I prepare pancakes at least once a week.It is a simple dish, fast and loved by children.You can serve them for any meal during the day.You can take pancakes to work, school or for a picnic.They are simply divine!',
 'El vidrio tiene una capacidad de 250 ml para mí.Para freír los panqueques, usé una sartén ancha a los panqueques: diámetro de 24 cm.Obtuve 16 panqueques muy delgados por la cantidad de ingredientes dados.Tome los huevos de la nevera de antemano.También puede calentar ligeramente la leche a una temper

In [134]:
import nltk
from nltk.tokenize import word_tokenize
import random

nltk.download('punkt_tab', quiet=True)
random.seed(42)

def safe_tokenize(text, language, verbose=False) -> list[str]:
    try:
        return word_tokenize(text, language=language)
    except Exception:
        if verbose:
            print(f"nltk tokenization possible for language [{language}]... Defaulting to whitespace tokenization")
        return text.split(" ")

tokenized = [safe_tokenize(t, lan, verbose=True) for t, lan in zip(translated, destinations.values())]

# replace random words with __MASKNOTTRANSLATED__
mask = "<mask>"
def apply_mask(text: list[str], num_masks=5):
    all_indicies = set(list(range(0, len(text)))) # so indexes will be unique
    for _ in range(num_masks):
        idx = random.choice(list(all_indicies))
        text[idx] = mask
        all_indicies -= {idx}
    return text

masked_splitted = []
for t in tokenized:
    masked_splitted.append(apply_mask(t))

In [135]:
for masked in masked_splitted:
    print(masked)

['The', 'glass', 'has', 'a', 'capacity', 'of', '<mask>', 'ml', 'for', 'me.To', 'fry', 'pancakes', ',', 'I', 'used', 'a', 'wide', 'pan', 'to', 'pancakes', ':', 'diameter', '24', 'cm.I', 'got', '16', 'very', 'thin', '<mask>', 'from', '<mask>', 'amount', 'of', '<mask>', 'given.Take', 'the', 'eggs', 'from', 'the', 'fridge', 'beforehand.You', 'can', 'also', 'lightly', 'heat', 'the', 'milk', 'to', 'a', 'temperature', 'not', 'higher', 'than', '40', 'degrees.Calories', 'were', 'counted', 'on', 'the', 'basis', 'of', 'the', 'ingredients', 'used', 'by', 'me.So', 'this', 'is', 'an', 'approximate', 'amount', 'of', '<mask>', ',', 'because', 'even', 'flour', 'can', 'have', 'a', 'different', 'amount', 'of', 'calories', 'than', 'the', 'one', 'I', 'used.I', 'prepare', 'pancakes', 'at', 'least', 'once', 'a', 'week.It', 'is', 'a', 'simple', 'dish', ',', 'fast', 'and', 'loved', 'by', 'children.You', 'can', 'serve', 'them', 'for', 'any', 'meal', 'during', 'the', 'day.You', 'can', 'take', 'pancakes', 'to', '

### Masking, Translation and Recreation of masked tokens

In [136]:
# some additional dependencies
!pip install -q protobuf sacremoses

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
3780.29s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [137]:
def fill_all_masks(sentence, mask_pipeline):
    placeholder = "UNIQUE_MASK_PLACEHOLDER"
    while mask in sentence:
        parts = sentence.split(mask, 1)
        rest = parts[1].replace(mask, placeholder)
        sentence_single_mask = parts[0] + mask + rest

        suggestions = mask_pipeline(sentence_single_mask)
        best_token = suggestions[0]['token_str']

        sentence_filled = sentence_single_mask.replace(mask, best_token, 1)
        sentence = sentence_filled.replace(placeholder, mask)
    return sentence

In [138]:
from transformers import pipeline
from googletrans import Translator

mask_pipeline = pipeline('fill-mask', model='allegro/herbert-base-cased')
translator = Translator()

for foreign_text, lan in zip(translated, destinations.keys()):
    # translate to HERBERT native language - polish
    pl_text = translator.translate(foreign_text, dest="pl").text
    # tokenize
    pl_tokens = safe_tokenize(pl_text, language=lan)
    # apply mask
    masked_tokens = apply_mask(pl_tokens)
    # join splitted
    masked = " ".join(masked_tokens)
    print("\nMasked: ", masked)
    
    final_sentence = fill_all_masks(masked, mask_pipeline)
    print("Filled: ", final_sentence)


Device set to use cpu



Masked:  Szkło ma dla mnie pojemność 250 ml. Do smażenia naleśników użyłem <mask> patelni <mask> <mask> średnica 24 cm. I dostałem 16 bardzo cienkich naleśników z ilości podanych składników. Wczoraj jaja z lodówki.Może mieć inną ilość kalorii niż ta, której użyłem. Przynajmniej przygotowuję naleśniki przynajmniej raz w tygodniu. To <mask> danie, <mask> i kochane przez dzieci. Możesz podać je na każdy posiłek w ciągu dnia. Możesz zabrać naleśniki do pracy, szkoły lub na piknik. Są po prostu boskie!
Filled:  Szkło ma dla mnie pojemność 250 ml. Do smażenia naleśników użyłem specjalnej patelni , jej średnica 24 cm. I dostałem 16 bardzo cienkich naleśników z ilości podanych składników. Wczoraj jaja z lodówki.Może mieć inną ilość kalorii niż ta, której użyłem. Przynajmniej przygotowuję naleśniki przynajmniej raz w tygodniu. To ulubione danie, popularne i kochane przez dzieci. Możesz podać je na każdy posiłek w ciągu dnia. Możesz zabrać naleśniki do pracy, szkoły lub na piknik. Są po prostu 

## Recursive translation

In [157]:
original_text = "Als Zweiter Weltkrieg wird der zweite global geführte Krieg sämtlicher Großmächte im 20. Jahrhundert bezeichnet. Über 60 Staaten waren direkt oder indirekt beteiligt, mehr als 110 Millionen Menschen trugen Waffen. Schätzungen zufolge wurden über 65 Millionen Menschen getötet."

In [158]:
destinations = {
    "en": "english",
    "es": "spanish",
    "lt": "lithuanian",
    "ru": "russian",
    "hu": "hungarian",
    "it": "italian",
    "fr": "french",
    "sl": "slovenian",
    "no": "norwegian",
    "de": "german"
}

In [163]:
from googletrans import Translator
from nltk.metrics import edit_distance
import copy

translator = Translator()
translated = copy.copy(original_text)
current_language = "de"
for lan in destinations.keys():
    print(f"Translating from '{current_language}' to '{lan}'")
    translated = translator.translate(translated, dest=lan, src="auto").text
    print(f"    Translated: {translated}")

    distance = edit_distance(original_text, translated, substitution_cost=1, transpositions=False)
    print(f"    Levenshtein edit distance: {distance}")
    current_language = lan

Translating from 'de' to 'en'
    Translated: The second global war of all major powers in the 20th century was called the Second World War.Over 60 states were directly or indirectly involved, more than 110 million people wore weapons.It is estimated that over 65 million people were killed.
    Levenshtein edit distance: 183
Translating from 'en' to 'es'
    Translated: La Segunda Guerra Global de todas las potencias importantes en el siglo XX se llamó la Segunda Guerra Mundial. Over 60 estados estaban directa o indirectamente involucrados, más de 110 millones de personas llevaban armas. Se estima que más de 65 millones de personas fueron asesinadas.
    Levenshtein edit distance: 207
Translating from 'es' to 'lt'
    Translated: Antrasis visų svarbių XX amžiaus galių karas buvo vadinamas Antrojo pasaulinio karo.Daugiau nei 60 valstijų tiesiogiai ar netiesiogiai dalyvavo, daugiau nei 110 milijonų žmonių nešiojo ginklus.Manoma, kad žuvo daugiau nei 65 milijonai žmonių.
    Levenshtein e

### How far is the translation and the original text?

In [167]:
print(f"Original: \n{original_text}")
print(f"Translated: \n{translated}")
distance = edit_distance(original_text, translated, substitution_cost=1, transpositions=False)
print(f"\nLevenshtein edit distance: {distance}")

Original: 
Als Zweiter Weltkrieg wird der zweite global geführte Krieg sämtlicher Großmächte im 20. Jahrhundert bezeichnet. Über 60 Staaten waren direkt oder indirekt beteiligt, mehr als 110 Millionen Menschen trugen Waffen. Schätzungen zufolge wurden über 65 Millionen Menschen getötet.
Translated: 
Der zweite Krieg für alle wichtigen Mächte in den 1900er Jahren wurde als Zweiten Weltkrieg bezeichnet. Hos 110 Millionen Menschen wurden direkt oder indirekt gemacht.

Levenshtein edit distance: 176


The score is high which means there are significant differences between original and multi-tranlated text.

## Translating pdf file

In [ ]:
!pip install -q PyPDF2 fpdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
9338.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40758 sha256=677026b363189e625a9afe030919a615a74b3f2427a28e5db8c4ed075137efed
  Stored in directory: /home/marcin/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [2]:
import PyPDF2
from fpdf import FPDF
from googletrans import Translator

def extract_text(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text() + "\n"
    return full_text

def translate(text, translate_to = "de"):
    translator = Translator()
    return translator.translate(text, dest=translate_to, src="en").text

# 3. Save text to PDF using fpdf (simpler than reportlab)
def save_text_to_pdf(text, output_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)
    for line in text.split('\n'):
        pdf.cell(0, 10, txt=line, ln=True)
    pdf.output(output_path)

# Usage
input_pdf = "EN-rival600-manu.pdf"
output_pdf = "DE-rival600-manu.pdf"

text = extract_text(input_pdf)
translated_text = translate(text)
save_text_to_pdf(translated_text, output_pdf)

If you look at the original file you can see that all syle was lost

## Translation with context awareness and large documents handling

In [ ]:
# [x] Find large document
# [x] Load document
# [x] Read full text
# [x] Merge text
# [x] Split into 10 sentenses long chunks and save 3 previous and 3 following sentences as context
# [] Use mBART or T5 to translate with surronding context
# [] Save results in 2 markdown files. One for each language. Each sentence in one

import PyPDF2
from fpdf import FPDF
from nltk.tokenize import word_tokenize, PunktSentenceTokenizer

class Chunk:
    def __init__(self, core, previous="", following=""):
        self.previous: str = previous
        self.core: str = core
        self.following: str = following

    @staticmethod
    def from_raw_text(corpus: str, chunk_size: int , context_size: int):
        assert chunk_size > context_size, "Chunk size must be grater than context"
        tokenizer = PunktSentenceTokenizer(corpus)
        sentences = tokenizer.tokenize(corpus)

        chunks: list['Chunk'] = []

        chunks_num = int(len(sentences)/chunk_size)
        remaining = len(sentences) - chunks_num

        for i in range(chunks_num):
            start_idx = i * chunk_size
            core = sentences[start_idx:start_idx + chunk_size]
            previous_context = sentences[start_idx-context_size: start_idx] if i != 0 else [""]
            following_context = sentences[start_idx: start_idx + context_size] if i < chunks_num else sentences[start_idx + context_size: len(sentences)]
            
            chunks.append(Chunk(Chunk.join_str(core), Chunk.join_str(previous_context), Chunk.join_str(following_context)))

        if remaining:
            previous_context = chunks[-1].following
            core = sentences[-3:]

            chunks.append(Chunk(core=core, previous=Chunk.join_str(previous_context), following=""))

        return chunks
    
    def __repr__(self) -> str:
        return f"{60*'='}\nCORE: {self.core}\nCONTEXT: \n\t{self.previous}\n\t+++++\n\t{self.following}\n{60*'='}"
    
    @staticmethod
    def join_str(splitted: list[str]):
        return " ".join(splitted)
    
text = extract_text("phd_thesis.pdf").replace('\n', '')

chunks = Chunk.from_raw_text(text, 10, 3)
chunks[10:12]

[============================================================
 CORE: Struktura rozprawy doktorskiej  zawiera części: teoretyczną oraz empiryczną. Dysertacja obejmuje wstęp, cztery rozdziały, zakończenie, bibliogra fię, spis rysunków, spis tabel i załączników. W rozdziale pierwszym pt. "Metodologia badań własnych", została podjęta t e-matyka przedmiotu ora z celu badań. Przedstawiony został problem badawczy, hipot e-zy robocze oraz hipotezy szczegółowe. Zostały poruszone kwestie metod, technik  oraz narzędzi badawczych. Zaprezentowano również obszar badań i scharakteryzow a-no próbę badawczą. Drugi rozdział  pt. "System bezpieczeństwa informacji w organizacji – teoria problemu" został poświęcony istocie systemu bezpieczeństwa informacji w organiz a-cji. W rozdziale tym zdefiniowano informację, przedstawiono istotę bezpieczeństwa informacji oraz opisano, czym jest b ezpieczeństwo informacji. Przedstawiono tu ró w-nież prawne wymagania, co do bezpieczeństwa informacji oraz środki technicz

## Translation with context awareness, tone adjustment and sentiment analysis

## Statistic translation with SMT and performence comparison